In [4]:
# Import necessary packages
import geopandas as gpd
import osmnx as ox

place_name = "Youssoufia, Morocco"
G = ox.graph_from_place(place_name, network_type='all_private')

gdf = gpd.read_file('map.geojson')

In [5]:
# Convert the graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [6]:
gdf_nodes.drop(gdf_nodes[gdf_nodes['highway'].isnull()].index, inplace=True)

In [64]:
gdf_nodes['geometry'][1109007239].coords

In [69]:
import folium
from folium.features import ClickForMarker, Tooltip
# Importing the geodesic module from the library
from geopy.distance import geodesic

# Loading the lat-long data for Kolkata & Delhi
kolkata = gdf_nodes['geometry'][1109007239].coords
delhi = point_dicts['mini_roundabout']

# Print the distance calculated in km
print(geodesic(kolkata, delhi).km)

6253.675183772957


In [42]:
# Capture click event

def print_hello(event):
    print('hello point', event.popup)

In [96]:
# Créer une carte centrée sur Youssoufia
map_youssoufia = folium.Map(location=[32.23630130013652, -8.51364643783117], zoom_start=13)

point_dicts = {}

# Ajouter les nœuds sur la carte
for idx, node in gdf_nodes.iterrows():
    marker = folium.Marker(
        [node.geometry.y, node.geometry.x], 
        popup=node['highway']
    )
    marker.add_to(map_youssoufia)

for x, y in gdf.iterrows():
    if x >= 2:
        color = y['marker-color']
        folium.Marker(location=[y.geometry.y, y.geometry.x], popup=y['name'], icon=folium.Icon(color=color)).add_to(map_youssoufia)
    else :
        folium.PolyLine(locations=y.geometry.coords, color='blue').add_to(map_youssoufia)

# Ajouter les arêtes sur la carte
for idx, edge in gdf_edges.iterrows():
    folium.PolyLine(locations=edge.geometry.coords, color='green').add_to(map_youssoufia)

# Listen for click events
# map_youssoufia.add_child(ClickForMarker("<b>Lat:</b> ${lat}<br /><b>Lon:</b> ${lng}"))

# Add latlong folium
folium.LatLngPopup().add_to(map_youssoufia)

/tmp/ipykernel_9971/731389954.py:17: UserWarning: color argument of Icon should be one of: {'orange', 'pink', 'beige', 'purple', 'darkblue', 'darkpurple', 'green', 'darkred', 'lightblue', 'black', 'gray', 'white', 'lightgreen', 'red', 'cadetblue', 'lightgray', 'blue', 'darkgreen', 'lightred'}.
  folium.Marker(location=[y.geometry.y, y.geometry.x], popup=y['name'], icon=folium.Icon(color=color)).add_to(map_youssoufia)


In [106]:
# Find popup slice
def find_popup_slice(html):
    '''
    Find the strating and ending index of popup function
    '''

    pattern = 'function latLngPop(e)'

    # Starting index
    starting_index = html.find(pattern)

    # Ending index
    tmp_html = html[starting_index:]

    # Look over the tmp html
    found = 0
    index = 0
    openning_found = False
    while not openning_found or found > 0:
        if tmp_html[index] == '{':
            found += 1
            openning_found = True
        elif tmp_html[index] == '}':
            found -= 1

        index += 1

    # Determine the ending index
    ending_index = starting_index + index

    return starting_index, ending_index

In [133]:
# Find map name
def find_map_variable_name(html):
    pattern = 'var map_'

    starting_index = html.find(pattern) + 4
    tmp_html = html[starting_index:]
    ending_index = tmp_html.find(' =') + starting_index

    return html[starting_index:ending_index]

# Find popup name
def find_popup_variable_name(html):
    pattern = 'var lat_lng'

    starting_index = html.find(pattern) + 4
    tmp_html = html[starting_index:]
    ending_index = tmp_html.find(' =') + starting_index

    return html[starting_index:ending_index]

In [146]:
# Define custom code function
def custom_code(popup_variable_name, map_variable_name):
    return '''
        // Custom code
        function latLngPop(e) {
            %s
                .setLatLng(e.latlng)
                .setContent(`
                    lat: ${e.latlng.lat}, lng: ${e.latlng.lng}
                    <button onClick="

                        fetch('http://localhost:3001', {
                            method: 'POST',
                            mode: 'no-cors',
                            headers: {
                                'Accept': 'application/json',
                                'Content-Type': 'application/json'
                            },
                            body: JSON.stringify({
                                latitude: e.latlng.lat,
                                longitude: e.latlng.lng
                            })
                        });

                        L.marker(
                            [e.latlng.lat, e.latlng.lng],
                            {}
                        ).addTo(%s);
                    "> 
                    Save coordinate
                    </button>
                    <button onClick="
                        fetch('http://localhost:3001', {
                            method: 'POST',
                            mode: 'no-cors',
                            headers: {
                                'Accept': 'application/json',
                                'Content-Type': 'application/json'
                            },
                            body: 'q'
                        });
                    ">
                        Quit
                    </button>
                `).openOn(%s);
        }
        // End custom code
    '''% (popup_variable_name, map_variable_name, map_variable_name)

In [144]:
# Create folium map 
def create_folium_map(map_file, center_coord):
    
    # Create a folium map
    map_youssoufia = folium.Map(location=center_coord, zoom_start=13)

    # Add popup
    folium.LatLngPopup().add_to(map_youssoufia)

    # Save the map to html
    map_youssoufia.save(map_file)

    # Read the folium file
    html = None
    with open(map_file, 'r') as mapfile:
        html = mapfile.read()

    # Determine popup function indecises
    pstart, pend = find_popup_slice(html) 

    # Find map variable name
    map_variable_name = find_map_variable_name(html)

    # Find map variable name
    popup_variable_name = find_popup_variable_name(html)

    # Inject code
    with open(map_file, 'w') as mapfile:
        mapfile.write(
            html[:pstart] +\
            custom_code(popup_variable_name, map_variable_name)+\
            html[pend:]
        )

In [151]:
# Import selenium packages
from selenium import webdriver

In [154]:
# Open the folium map selenium
def open_folium_map(project_url, map_file):
    driver = None
    try:
        driver = webdriver.Firefox()
        driver.get(
            project_url = map_file
        )
    except Exception as ex:
        print('Driver failed ton open/find url: ', ex)

    return driver

In [156]:
center_coord = [32.23630130013652, -8.51364643783117]
map_file = 'map_youssoufia.html'

# Create the folium map
create_folium_map(map_file, center_coord)

# Open the folium map selenium
project_map_url = 'file:///home/like/Documents/Projects/Youcode/Challenges/city_map_points/'
driver = open_folium_map(project_map_url, map_file)
import time

# Run webserver that listen to sent coordinates

# Close the folium map

# Print all collected coords

Driver failed ton open/find url:  Message: Service /snap/bin/geckodriver unexpectedly exited. Status code was: -6

